In [ ]:
from WikilinksIterator import WikilinksNewIterator
from DbWrapper import WikipediaDbWrapper

input_dir = "data/enwiki/extracted"
output_dir = "data/intralinks/all"
db_user = "user"
db_pass = "passcode"
db_schema = "NEDforNoisyText"
db_host = "localhost"
wikiDB = WikipediaDbWrapper(user=db_user, password=db_pass, database=db_schema, host=db_host)

writer = wlink_writer(output_dir + '_tmp')
for mention, target, left_context, right_context in wikipedia_crossref_iterator(input_dir):
    wlink = dict()
    wlink['wikiurl'] = target
    wlink['word'] = mention
    wlink['left_context'] = left_context
    wlink['right_context'] = right_context
    writer.save(wlink)
writer.finalize()

it = WikilinksNewIterator(output_dir + '_tmp', resolveIds=True, db=wikiDB)
writer = wlink_writer(output_dir)
for wlink in it.jsons():
    writer.save(wlink)
writer.finalize()

In [ ]:
from WikilinksIterator import WikilinksNewIterator
import os
from PPRforNED import *
from DbWrapper import WikipediaDbWrapper

wikipedia_corpus_dir = 'data/intralinks/all'

print 'calculate wikipedia corpus statistics'
it = WikilinksNewIterator(wikipedia_corpus_dir)
stats = WikilinksStatistics(it)
stats.calcStatistics()
stats.saveToFile(os.path.join(wikipedia_corpus_dir, 'all_stats'))
stats.printSomeStats()

print 'calculate PPRforNED stats'
ppr_itr = sPPRIterator(path='data/PPRforNED/AIDA_candidates')
ppr_stats = PPRStatistics(ppr_itr)
wikiDB = WikipediaDbWrapper(user=db_user, password=db_pass, database=db_schema, host=db_host)
ppr_stats.calcStatistics(wikiDB)
print len(ppr_stats.conceptNames)
ppr_stats.saveToFile('/data/PPRforNED/ppr_stats')

In [ ]:
experiment_path = "experiments/pretrainWikipedia/experiment.conf"
experiment = Experiment(db_user, db_pass, db_schema, db_host, experiment_path)
for x in xrange(8):
    trained_mentions = experiment.train(model_name='.'+str(x))
    experiment.evaluate()

In [ ]:
experiment_path = "experiments/CoNLL/experiment.conf"
experiment = Experiment(db_user, db_pass, db_schema, db_host, experiment_path)
for x in xrange(8):
    trained_mentions = experiment.train(model_name='.'+str(x))
    experiment.evaluate()